In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from functions.Pre_processing_Casos_Graves import *
from functions.Pre_processing_models import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

from sklearn.model_selection import RepeatedKFold

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Lendo o Dataset:

In [3]:
df = pd.read_csv(r'C:\Users\gabri\Desktop\Git - Tcc\Estudo_dados_de_covid\Data\Casos Graves\df_casos_graves')

### Utilizando a class 'random forest' para fazer o pré-processamento:

In [4]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino','sexo_Masculino', 'data_notificacao']

In [5]:
Pre_processing_models = Pre_processing_models(df)
Pre_processing_models.run_graves()

df = Pre_processing_models.df
df = df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino,data_notificacao
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0.460870,0,1,2020-05-30
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,1,0,2020-06-02
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.747826,1,0,2020-06-02
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.573913,0,1,2020-06-01
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.591304,0,1,2020-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.669565,0,1,2022-06-08
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,1,2022-06-09
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.643478,0,1,2022-06-10
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.634783,1,0,2022-06-12


In [6]:
df2 = pd.DataFrame()
df2 = df
df2.set_index('data_notificacao', inplace=True)
df2.sort_index(inplace=True)
df2

C:\Users\gabri\AppData\Local\Temp\ipykernel_560\943203674.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_index(inplace=True)


,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
data_notificacao,,,,,,,,,,,,,,,,,,,,,
2020-02-26,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.356522,0,1
2020-02-26,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.278261,0,1
2020-02-27,1,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0.217391,1,0
2020-02-27,1,0,1,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.278261,0,1
2020-02-27,1,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.260870,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-12,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,1,0
2022-06-12,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.556522,0,1
2022-06-12,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.573913,1,0


### Criação de uma função para instância do modelo, utilizando repeated K-fold:

In [12]:
from sklearn.metrics import f1_score, make_scorer

def KNN_model (X_train, y_train):
    k_range = range(1, 31)
    param_grid = {"n_neighbors": k_range}
    #scores = ['accuracy','f1','precision','recall','roc_auc']

    f1 = make_scorer(f1_score , average='macro')

    #Init classifier
    knn = KNeighborsClassifier()

    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits = 5, n_repeats = 10, random_state = 1)

    # Init Grid Search
    grid_cv = GridSearchCV(knn, param_grid, n_jobs = -1, cv=5, scoring = 'roc_auc')
    #grid_cv = GridSearchCV(knn, param_grid, n_jobs = -1, cv=cv, scoring = scores, refit='accuracy')

    # Fit 
    grid_cv.fit(X_train, y_train)

    #mean_test_score = grid_cv.cv_results_['mean_test_score']
    #std_test_score = grid_cv.cv_results_['std_test_score']
    best_score = grid_cv.best_score_
    best_params = grid_cv.best_params_

   

    print('#'*40)
    #print(f"mean_test_score: {grid_cv.cv_results_['mean_test_score']}")
    #print(f"std_test_score: {grid_cv.cv_results_['std_test_score']}")
    print('')
    print(f'best_score: {grid_cv.best_score_}')
    print(f'best_params_: {grid_cv.best_params_}')
    print('#'*40)
    print('')

    #return mean_test_score, std_test_score, best_score, best_params
    return best_score, best_params

### Aplicação do modelo para diferentes 'random_states':

In [13]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 2)

#utilizando diferentes random_states para mudar os conjuntos de treino e teste
#list_random_states = [0,12,42,50,54,68,79,81,92,112]
list_random_states = [42]

#listas para armazenar os resultados
#mean_test_scores = []
#std_test_scores = []
best_scores = []
best_params = []

X = df2.drop(labels=['death'], axis=1)
y = df2['death']

# X = df2[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE', 'SYMPTOM_COLD', 'SYMPTOM_DIARRHEA','SYMPTOM_LOW_OXYGEN_SATURATION', 'SYMPTOM_BREATHING_CONDITION',
#     'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'DISEASE_KIDNEY', 'DISEASE_IMMUNOSUPPRESSION', 'DISEASE_ETILISM', 'DISEASE_LIVER',
#     'sexo_Feminino','sexo_Masculino']]

# y = df2['death'].astype(int)

for random_state in list_random_states:

    for train_index, test_index in tss.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    #mean_test_scores.append(mean_test_score)
    #std_test_scores.append(std_test_score)
    best_scores.append(best_score)
    best_params.append(best_param)

########################################

best_score: 0.7927820131625383
best_params_: {'n_neighbors': 29}
########################################



## Testando o modelo para os melhores parâmetros:

In [7]:
print(f'Best_score = {max(best_scores)}')
index = best_scores.index(max(best_scores))
print(f'Best_params = {best_params[index]}')

Best_score = 0.7574310337470982
Best_params = {'n_neighbors': 30}


In [10]:
# parametrizando o modelo final
final_cl = KNeighborsClassifier(n_neighbors=30)

#treinando o modelo final
final_cl.fit(X_train, y_train)


preds = final_cl.predict(X_test)

accuracy_score(y_test, preds)

0.7378893920680665

## Aplicando para os períodos de 6 meses:

In [20]:
df = pd.read_csv(r'C:\Users\mcacs\Desktop\Estudo_dados_de_covid\Data\Casos Graves\df_casos_graves')

In [21]:
periodo_1 = df[(df['data_notificacao'] > '2020-02-26') & (df['data_notificacao'] < '2020-08-31')]
periodo_2 = df[(df['data_notificacao'] > '2020-09-01') & (df['data_notificacao'] < '2021-02-28')]
periodo_3 = df[(df['data_notificacao'] > '2021-03-01') & (df['data_notificacao'] < '2021-08-31')]
periodo_4 = df[(df['data_notificacao'] > '2021-09-01') & (df['data_notificacao'] < '2022-02-28')]
periodo_5 = df[(df['data_notificacao'] > '2022-03-01') & (df['data_notificacao'] < '2022-06-13')]

#### Criando listas para armazenamento dos melhores parâmetros:

In [22]:
best_scores_periodos_6meses = []
best_params_periodos_6meses = []

#### Período 1 (26/02/2020 até 31/08/2020):

In [60]:
random_forest = Random_forest(periodo_1)
random_forest.run_graves()

df_p1 = random_forest.df
df_p1 = df_p1[list_features]
df_p1

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.486239,0,1
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.788991,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.605505,0,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.623853,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36273,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0.211009,1,0
36274,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.614679,0,1
36275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.623853,0,1
36276,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.752294,1,0


In [61]:
#utilizando diferentes random_states para mudar os conjuntos de treino e teste
list_random_states = [0,12,42,50,54,68,79,81,92,112]

#listas para armazenar os resutados
best_scores_1 = []
best_params_1 = []

X = df_p1[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE',
    'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'idade', 'sexo_Feminino','sexo_Masculino']]

y = df_p1['death'].astype(int)

for random_state in list_random_states:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    best_scores_1.append(best_score)
    best_params_1.append(best_param)


#armazenando as informações na lista para todos os periodos de 6 meses
best_scores_periodos_6meses.append(best_scores_1)
best_params_periodos_6meses.append(best_params_1)

########################################

best_score: 0.7973500957477506
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7985643987420175
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7983200656389451
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.8028714386063092
best_params_: {'n_neighbors': 29}
########################################

########################################

best_score: 0.8010689587832894
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.8002368452500661
best_params_: {'n_neighbors': 20}
########################################

########################################

best_score: 0.8048793490901778
best_params_: {'n_neighbors': 30}

In [64]:
print(f'Best_score = {max(best_scores_periodos_6meses[0])}')
index = best_scores_periodos_6meses[0].index(max(best_scores_periodos_6meses[0]))
print(f'Best_params = {best_params_periodos_6meses[0][index]}')

Best_score = 0.8048793490901778
Best_params = {'n_neighbors': 30}


## Testando o modelo para os melhores parâmetros:

In [65]:
# parametrizando o modelo final
final_cl_p1 = KNeighborsClassifier(n_neighbors=30)

#treinando o modelo final
final_cl_p1.fit(X_train, y_train)


preds = final_cl_p1.predict(X_test)

accuracy_score(y_test, preds)

0.8018320610687023

#### Período 2 (01/09/2020 até 28/02/2021):

In [66]:
random_forest = Random_forest(periodo_2)
random_forest.run_graves()

df_p2 = random_forest.df
df_p2 = df_p2[list_features]
df_p2

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
431,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0.736842,0,1
432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.166667,0,1
433,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.798246,1,0
434,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.500000,0,1
436,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.736842,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36245,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.666667,1,0
36246,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.570175,1,0
36247,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.429825,0,1
36351,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0.798246,1,0


In [67]:
#utilizando diferentes random_states para mudar os conjuntos de treino e teste
list_random_states = [0,12,42,50,54,68,79,81,92,112]

#listas para armazenar os resutados
best_scores_2 = []
best_params_2 = []

X = df_p2[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE',
    'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'idade', 'sexo_Feminino','sexo_Masculino']]

y = df_p2['death'].astype(int)

for random_state in list_random_states:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    best_scores_2.append(best_score)
    best_params_2.append(best_param)


#armazenando as informações na lista para todos os periodos de 6 meses
best_scores_periodos_6meses.append(best_scores_2)
best_params_periodos_6meses.append(best_params_2)


########################################

best_score: 0.7262629302752182
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7276180352663195
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.7275695638177035
best_params_: {'n_neighbors': 24}
########################################

########################################

best_score: 0.7267563611183663
best_params_: {'n_neighbors': 24}
########################################

########################################

best_score: 0.7288987627024959
best_params_: {'n_neighbors': 26}
########################################

########################################

best_score: 0.7261138714595493
best_params_: {'n_neighbors': 24}
########################################

########################################

best_score: 0.7232576700904437
best_params_: {'n_neighbors': 30}

In [69]:
print(f'Best_score = {max(best_scores_periodos_6meses[1])}')
index = best_scores_periodos_6meses[1].index(max(best_scores_periodos_6meses[1]))
print(f'Best_params = {best_params_periodos_6meses[1][index]}')

Best_score = 0.7291189797914148
Best_params = {'n_neighbors': 27}


## Testando o modelo para os melhores parâmetros:

In [70]:
# parametrizando o modelo final
final_cl_p2 = KNeighborsClassifier(n_neighbors=27)

#treinando o modelo final
final_cl_p2.fit(X_train, y_train)


preds = final_cl_p2.predict(X_test)

accuracy_score(y_test, preds)

0.7201970443349753

#### Período 3 (01/03/2021 até 31/08/2021):

In [71]:
random_forest = Random_forest(periodo_3)
random_forest.run_graves()

df_p3 = random_forest.df
df_p3 = df_p3[list_features]
df_p3

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
952,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.429825,0,1
953,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.315789,1,0
954,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.368421,1,0
955,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.394737,0,1
956,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.377193,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36364,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.622807,1,0
36366,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0.508772,1,0
36371,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.403509,0,1
36372,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.622807,0,1


In [72]:
#utilizando diferentes random_states para mudar os conjuntos de treino e teste
list_random_states = [0,12,42,50,54,68,79,81,92,112]

#listas para armazenar os resutados
best_scores_3 = []
best_params_3 = []

X = df_p3[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE',
    'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'idade', 'sexo_Feminino','sexo_Masculino']]

y = df_p3['death'].astype(int)

for random_state in list_random_states:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    best_scores_3.append(best_score)
    best_params_3.append(best_param)


#armazenando as informações na lista para todos os periodos de 6 meses
best_scores_periodos_6meses.append(best_scores_3)
best_params_periodos_6meses.append(best_params_3)

########################################

best_score: 0.7417433444922685
best_params_: {'n_neighbors': 20}
########################################

########################################

best_score: 0.7376549677830744
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.7385313812910423
best_params_: {'n_neighbors': 26}
########################################

########################################

best_score: 0.7386174233658845
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7375264394769185
best_params_: {'n_neighbors': 20}
########################################

########################################

best_score: 0.7424042414238184
best_params_: {'n_neighbors': 26}
########################################

########################################

best_score: 0.7399794716732745
best_params_: {'n_neighbors': 30}

In [74]:
print(f'Best_score = {max(best_scores_periodos_6meses[2])}')
index = best_scores_periodos_6meses[2].index(max(best_scores_periodos_6meses[2]))
print(f'Best_params = {best_params_periodos_6meses[2][index]}')

Best_score = 0.7424042414238184
Best_params = {'n_neighbors': 26}


## Testando o modelo para os melhores parâmetros:

In [75]:
# parametrizando o modelo final
final_cl_p3 = KNeighborsClassifier(n_neighbors=26)

#treinando o modelo final
final_cl_p3.fit(X_train, y_train)


preds = final_cl_p3.predict(X_test)

accuracy_score(y_test, preds)

0.7401032702237521

#### Período 4 (01/09/2021 até 28/02/2022):

In [76]:
random_forest = Random_forest(periodo_3)
random_forest.run_graves()

df_p4 = random_forest.df
df_p4 = df_p4[list_features]
df_p4

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
952,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.429825,0,1
953,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.315789,1,0
954,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.368421,1,0
955,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.394737,0,1
956,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.377193,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36364,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.622807,1,0
36366,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0.508772,1,0
36371,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.403509,0,1
36372,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.622807,0,1


In [77]:
#utilizando diferentes random_states para mudar os conjuntos de treino e teste
list_random_states = [0,12,42,50,54,68,79,81,92,112]

#listas para armazenar os resutados
best_scores_4 = []
best_params_4 = []

X = df_p4[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE',
    'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'idade', 'sexo_Feminino','sexo_Masculino']]

y = df_p4['death'].astype(int)

for random_state in list_random_states:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    best_scores_4.append(best_score)
    best_params_4.append(best_param)


#armazenando as informações na lista para todos os periodos de 6 meses
best_scores_periodos_6meses.append(best_scores_4)
best_params_periodos_6meses.append(best_params_4)

########################################

best_score: 0.7417433444922685
best_params_: {'n_neighbors': 20}
########################################

########################################

best_score: 0.7376549677830744
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.7385313812910423
best_params_: {'n_neighbors': 26}
########################################

########################################

best_score: 0.7386174233658845
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7375264394769185
best_params_: {'n_neighbors': 20}
########################################

########################################

best_score: 0.7424042414238184
best_params_: {'n_neighbors': 26}
########################################

########################################

best_score: 0.7399794716732745
best_params_: {'n_neighbors': 30}

In [78]:
print(f'Best_score = {max(best_scores_periodos_6meses[3])}')
index = best_scores_periodos_6meses[3].index(max(best_scores_periodos_6meses[3]))
print(f'Best_params = {best_params_periodos_6meses[3][index]}')

Best_score = 0.7424042414238184
Best_params = {'n_neighbors': 26}


## Testando o modelo para os melhores parâmetros:

In [79]:
# parametrizando o modelo final
final_cl_p4 = KNeighborsClassifier(n_neighbors=26)

#treinando o modelo final
final_cl_p4.fit(X_train, y_train)


preds = final_cl_p4.predict(X_test)

accuracy_score(y_test, preds)

0.7401032702237521

#### Período 5 (01/03/2022 até 13/06/2022):

In [23]:
random_forest = Random_forest(periodo_5)
random_forest.run_graves()

df_p5 = random_forest.df
df_p5 = df_p5[list_features]
df_p5

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino,data_notificacao
2165,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0.281818,0,1,2022-03-02
2169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.600000,1,0,2022-03-07
2172,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0.554545,0,1,2022-03-12
2182,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0.554545,0,1,2022-03-14
2185,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0,1,2022-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.700000,0,1,2022-06-08
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.672727,0,1,2022-06-09
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.672727,0,1,2022-06-10
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.663636,1,0,2022-06-12


In [24]:
df2_p5 = pd.DataFrame()
df2_p5 = df_p5
df2_p5.set_index('data_notificacao', inplace=True)
df2_p5.sort_index(inplace=True)
df2_p5

c:\Users\mcacs\anaconda3\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().sort_index(


,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
data_notificacao,,,,,,,,,,,,,,,,,,,,,
2022-03-02,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.281818,0,1
2022-03-02,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.636364,1,0
2022-03-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.681818,0,1
2022-03-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.836364,0,1
2022-03-02,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0.427273,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-12,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,1,0
2022-06-12,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0,1
2022-06-12,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.581818,0,1


In [25]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 2)

#utilizando diferentes random_states para mudar os conjuntos de treino e teste
#list_random_states = [0,12,42,50,54,68,79,81,92,112]
list_random_states = [42]

#listas para armazenar os resutados
best_scores_5 = []
best_params_5 = []

X = df2_p5.drop(labels=['death'], axis=1)
y = df2_p5['death']

"""X = df_p5[["SYMPTOM_COUGH", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", 'SYMPTOM_THROAT_CONDITION', 'SYMPTOM_HEADACHE',
    'DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY', 'DISEASE_TABAGIST', 'idade', 'sexo_Feminino','sexo_Masculino']]

y = df_p5['death'].astype(int)"""

for random_state in list_random_states:
    for train_index, test_index in tss.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    best_scores_5.append(best_score)
    best_params_5.append(best_param)


#armazenando as informações na lista para todos os periodos de 6 meses
best_scores_periodos_6meses.append(best_scores_5)
best_params_periodos_6meses.append(best_params_5)

########################################

best_score: nan
best_params_: {'n_neighbors': 1}
########################################



c:\Users\mcacs\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [26]:
X_train.index

Index(['2022-03-02', '2022-03-02', '2022-03-02', '2022-03-02', '2022-03-02',
       '2022-03-02', '2022-03-02', '2022-03-02', '2022-03-02', '2022-03-02',
       ...
       '2022-05-05', '2022-05-05', '2022-05-05', '2022-05-05', '2022-05-06',
       '2022-05-06', '2022-05-06', '2022-05-06', '2022-05-06', '2022-05-06'],
      dtype='object', name='data_notificacao', length=1109)

In [27]:
X_test.index

Index(['2022-05-06', '2022-05-06', '2022-05-07', '2022-05-07', '2022-05-07',
       '2022-05-07', '2022-05-07', '2022-05-08', '2022-05-08', '2022-05-08',
       ...
       '2022-06-11', '2022-06-11', '2022-06-12', '2022-06-12', '2022-06-12',
       '2022-06-12', '2022-06-12', '2022-06-12', '2022-06-12', '2022-06-12'],
      dtype='object', name='data_notificacao', length=554)

In [82]:
print(f'Best_score = {max(best_scores_periodos_6meses[4])}')
index = best_scores_periodos_6meses[4].index(max(best_scores_periodos_6meses[4]))
print(f'Best_params = {best_params_periodos_6meses[4][index]}')

Best_score = 0.7424042414238184
Best_params = {'n_neighbors': 26}


## Testando o modelo para os melhores parâmetros:

In [83]:
# parametrizando o modelo final
final_cl_p5 = KNeighborsClassifier(n_neighbors=26)

#treinando o modelo final
final_cl_p5.fit(X_train, y_train)


preds = final_cl_p5.predict(X_test)

accuracy_score(y_test, preds)

0.7401032702237521